In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from scipy.stats import skew
import numpy as np
import re
import os
from datetime import timedelta

### Create one dataframe containing all events for each resolution

In [2]:
# all_events_5mins =[]
# all_events_10mins =[]
# all_events_30mins =[]
# all_events_60mins =[]

# total_events = 0
# print(len(os.listdir("../DanishRainData_Outputs/5mins/")))

# for num, file in enumerate(np.sort(os.listdir("../DanishRainData_Outputs/5mins/"))):
#     if file != "All_events_571820_precip_minute.csv":
#         # Get 5 mintue data
#         df_5m = pd.read_csv(f"../DanishRainData_Outputs/5mins/{file}")
#         df_5m['event_num']=df_5m.index
#         # Get rid of the trailing strings sometimes on the gauge_num
#         df_5m['gauge_num'] = df_5m['gauge_num'].apply(
#             lambda x: int(x.split('_')[0]) if isinstance(x, str) and x.split('_')[0].isdigit() else x)

#         # Get other resolution data
#         df_10m = pd.read_csv(f"../DanishRainData_Outputs/{10}mins/{file}")
#         df_30m = pd.read_csv(f"../DanishRainData_Outputs/{30}mins/{file}")
#         df_60m = pd.read_csv(f"../DanishRainData_Outputs/{60}mins/{file}")

#        # Find common event numbers in all four
#         common_event_nums = (
#             set(df_5m['event_num']) &
#             set(df_10m['event_num']) &
#             set(df_30m['event_num']) &
#             set(df_60m['event_num']))

#         # Filter all DataFrames to only include common events
#         df_5m = df_5m[df_5m['event_num'].isin(common_event_nums)].reset_index(drop=True)
#         df_10m = df_10m[df_10m['event_num'].isin(common_event_nums)].reset_index(drop=True)
#         df_30m = df_30m[df_30m['event_num'].isin(common_event_nums)].reset_index(drop=True)
#         df_60m = df_60m[df_60m['event_num'].isin(common_event_nums)].reset_index(drop=True)

#         total_events = total_events + len(common_event_nums)
#         print(f"{num} : {file}: {len(common_event_nums)} events retained across all resolutions, new total: {total_events}")    

#         # Collect DataFrames into a dictionary for easy iteration
#         dfs = {'5m': df_5m,
#             '10m': df_10m,
#             '30m': df_30m,
#             '60m': df_60m}

#         # Check the start and end times align
#         for i in range(len(df_5m)):
#             reference_start = pd.to_datetime(df_5m.iloc[i]['start_time'])
#             reference_end = pd.to_datetime(df_5m.iloc[i]['end_time'])
#             # print(reference_start,reference_end)
#             for res, df in dfs.items():
#                 start = pd.to_datetime(df.iloc[i]['start_time'])
#                 end = pd.to_datetime(df.iloc[i]['end_time'])
#                 #print(res, start, end)

#                 delta_start = abs(start - reference_start)
#                 delta_end = abs(end - reference_end)

#                 if delta_start > timedelta(minutes=60):
#                     print(f"⚠️ START mismatch in {res} at index {i}")
#                     print(f"   {res} start: {start}, 5m start: {reference_start}, Δ: {delta_start}")

#                 if delta_end > timedelta(minutes=60):
#                     print(f"⚠️ END mismatch in {res} at index {i}")
#                     print(f"   {res} end: {end}, 5m end: {reference_end}, Δ: {delta_end}")

#         all_events_5mins.append(df_5m)
#         all_events_10mins.append(df_10m)             
#         all_events_30mins.append(df_30m)
#         all_events_60mins.append(df_60m)            

# all_events_df_5mins = pd.concat(all_events_5mins)   
# all_events_df_10mins = pd.concat(all_events_10mins)   
# all_events_df_30mins = pd.concat(all_events_30mins)
# all_events_df_60mins = pd.concat(all_events_60mins)   

# for df in [all_events_df_5mins, all_events_df_10mins, all_events_df_30mins, all_events_df_60mins]:
#     df.reset_index(drop=True, inplace=True)
    
# print(len(all_events_df_5mins), len(all_events_df_10mins), len(all_events_df_30mins), len(all_events_df_60mins))

In [3]:
# all_events_df_5mins.to_csv('Data/all_events_df_5mins.csv', index=False)
# all_events_df_10mins.to_csv('Data/all_events_df_10mins.csv', index=False)
# all_events_df_30mins.to_csv('Data/all_events_df_30mins.csv', index=False)
# all_events_df_60mins.to_csv('Data/all_events_df_60mins.csv', index=False)

### Scale and transform

In [4]:
all_events_df_5mins = pd.read_csv('Data/all_events_df_5mins.csv')
all_events_df_10mins = pd.read_csv('Data/all_events_df_10mins.csv')
all_events_df_30mins = pd.read_csv('Data/all_events_df_30mins.csv')
all_events_df_60mins = pd.read_csv('Data/all_events_df_60mins.csv')

In [5]:
# all_events_df_5mins = all_events_df_5mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})
# all_events_df_10mins = all_events_df_10mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})
# all_events_df_30mins = all_events_df_30mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})
# all_events_df_60mins = all_events_df_60mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})

# all_events_df_5mins = all_events_df_5mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})
# all_events_df_10mins = all_events_df_10mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})
# all_events_df_30mins = all_events_df_30mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})
# all_events_df_60mins = all_events_df_60mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})

### Delete columns we dont want

### Remove rows with unrealistically high precip

In [6]:
bad_indices = all_events_df_5mins.sort_values(by="total_precip", ascending=False)[:15].index

# Drop rows at these indices from all dataframes
all_events_df_5mins = all_events_df_5mins.drop(index=bad_indices)
all_events_df_10mins = all_events_df_10mins.drop(index=bad_indices)
all_events_df_30mins = all_events_df_30mins.drop(index=bad_indices)
all_events_df_60mins = all_events_df_60mins.drop(index=bad_indices)

for df in [all_events_df_5mins, all_events_df_10mins, all_events_df_30mins, all_events_df_60mins]:
    df.reset_index(drop=True, inplace=True)
    
print(len(all_events_df_5mins), len(all_events_df_10mins), len(all_events_df_30mins), len(all_events_df_60mins))

397157 397157 397157 397157


### Remove very small events

In [7]:
print(len(all_events_df_5mins))
all_events_df_5mins = all_events_df_5mins[all_events_df_5mins['total_precip']>4].copy()
print(len(all_events_df_5mins))

print(len(all_events_df_10mins))
all_events_df_10mins = all_events_df_10mins[all_events_df_10mins['total_precip']>4].copy()
print(len(all_events_df_10mins))

print(len(all_events_df_30mins))
all_events_df_30mins = all_events_df_30mins[all_events_df_30mins['total_precip']>4].copy()
print(len(all_events_df_30mins))

print(len(all_events_df_60mins))
all_events_df_60mins = all_events_df_60mins[all_events_df_60mins['total_precip']>4].copy()
print(len(all_events_df_60mins))

397157
240579
397157
240590
397157
240609
397157
240588


# Create version for all resolutions, just raw
### Specify just raw columns to keep

In [8]:
raw_cols = []
for col in all_events_df_5mins.columns:
    if not col.endswith('_DMC_10') and not col.endswith('dblnorm'):
        raw_cols.append(col)

### Also remove 'frac_q1', 'frac_q2', 'frac_q3', 'frac_q4'
Because these bring lots of nans (with current calculation method)  
ni is the same as peak mean ratio

In [9]:
cols_to_del = ['frac_q1', 'frac_q2', 'frac_q3', 'frac_q4', 'BSC', 'ni', 'm1', 'm2', 'm3', 'm4', 'm5']
more_cols_to_del = ['gauge_num',  'start_time', 'end_time', 'min_intensity', 'heaviest_half']
#                     'duration' ,'total_precip',  'event_num',
#                     'peak_mean_ratio_scaled',
                   
raw_cols = [x for x in raw_cols if x not in cols_to_del]  
raw_cols = [x for x in raw_cols if x not in more_cols_to_del]    

In [10]:
all_events_df_5mins_raw = all_events_df_5mins[raw_cols].copy()
all_events_df_10mins_raw = all_events_df_10mins[raw_cols].copy()
all_events_df_30mins_raw = all_events_df_30mins[raw_cols].copy()
all_events_df_60mins_raw = all_events_df_60mins[raw_cols].copy()

In [11]:
all_events_df_60mins_raw["I30"] = 0

In [12]:
print(len(all_events_df_5mins_raw), len(all_events_df_10mins_raw), len(all_events_df_30mins_raw), len(all_events_df_60mins_raw))

bad_indices_all = []
for all_events_df in [all_events_df_5mins_raw, all_events_df_10mins_raw, all_events_df_30mins_raw, all_events_df_60mins_raw]:
    columns_with_nan = all_events_df.columns[all_events_df.isnull().any()].tolist()
    print("Columns with at least one NaN value:")
    print(columns_with_nan)
    # #Find indices of rows with any NaNs in the 60-minute data
    bad_indices = all_events_df[all_events_df.isnull().any(axis=1)].index
    bad_indices_lst = bad_indices.tolist()
    print(len(bad_indices_lst))
    if len(bad_indices) >0:
        bad_indices_all.extend(bad_indices_lst)
    
# Drop rows at these indices from all dataframes
all_events_df_5mins_raw = all_events_df_5mins_raw.drop(index=bad_indices_all)
all_events_df_10mins_raw = all_events_df_10mins_raw.drop(index=bad_indices_all)
all_events_df_30mins_raw = all_events_df_30mins_raw.drop(index=bad_indices_all)
all_events_df_60mins_raw = all_events_df_60mins_raw.drop(index=bad_indices_all)   

print(len(all_events_df_5mins_raw), len(all_events_df_10mins_raw), len(all_events_df_30mins_raw), len(all_events_df_60mins_raw))

240579 240590 240609 240588
Columns with at least one NaN value:
[]
0
Columns with at least one NaN value:
[]
0
Columns with at least one NaN value:
[]
0
Columns with at least one NaN value:
['kurtosis', 'asymm_d']
7632
232947 232958 232977 232956


### Join all into one dataframe

In [13]:
metric_columns =all_events_df_5mins_raw.columns

In [14]:
all_events_df_5mins_raw = all_events_df_5mins_raw.add_suffix('_5m')
all_events_df_10mins_raw = all_events_df_10mins_raw.add_suffix('_10m')
all_events_df_30mins_raw = all_events_df_30mins_raw.add_suffix('_30m')
all_events_df_60mins_raw = all_events_df_60mins_raw.add_suffix('_60m')

In [15]:
all_events_all_res = pd.concat([all_events_df_5mins_raw, all_events_df_10mins_raw,
                               all_events_df_30mins_raw, all_events_df_60mins_raw], axis=1)
# Step 2: Define the event number columns from each resolution
event_cols = ['event_num_5m', 'event_num_10m', 'event_num_30m', 'event_num_60m']

# Step 3: Drop rows with any missing event numbers
all_events_all_res = all_events_all_res.dropna(subset=event_cols)

# Step 4: Keep only rows where all event numbers are equal (same event matched across resolutions)
all_events_all_res = all_events_all_res[all_events_all_res[event_cols].nunique(axis=1) == 1]

In [16]:
all_events_all_res.to_csv("Data/NotScaled_AllRes.csv",index=False)

# Create version for 5 minutes, DMC and raw
### Specify columns to keep

In [25]:
cols_to_del = ['frac_q1', 'frac_q2', 'frac_q3', 'frac_q4', 'BSC', 'ni', 'T50', 'm1', 'm2', 'm3', 'm4', 'm5', 'BSC_Index']
more_cols_to_del = ['gauge_num', 'start_time', 'end_time', 'duration', 'total_precip', 'event_num', 'heaviest_half']
all_to_del = cols_to_del + more_cols_to_del
all_columns = df.columns.tolist()
cols_to_keep = [col for col in all_columns if col not in all_to_del]

# Remove any column that matches exactly or starts with one of the names (e.g., has suffixes like _DMC_10)
cols_to_keep = [col for col in cols_to_keep if col not in all_to_del]

# Then filter the DataFrame
all_events_df_5mins_filtered = all_events_df_5mins[cols_to_keep].copy()

In [26]:
all_events_df_5mins_filtered['mean_intensity_DMC_10'] = all_events_df_5mins_filtered['mean_intensity_DMC_10']/10

In [27]:
all_events_df_5mins_filtered.to_csv("Data/NotScaled_RawVsDMC.csv",index=False)